## Description

Typical office material supply company running their operations sees many challenges, one of the major ones being how to measure the effectiveness of sales and marketing operations. We have picked up an imaginary office material supply company “Office Corp.” for our analysis. They have tested a telemarketing campaign targeting existing domestic business customers with a random selection of products. For this exercise, a sample of 16,172 customers were targeted. The products being marketed are Desk, Executive Chair, Standard Chair, Monitor, Printer Computer, Insurance, Toner and Office Supplies.  

Office Corp would like to leverage the knowledge gained from this survey (and associated dataset) for future campaigns. Office Corp has hired an analyst team to solve the following three problems using various analytical techniques.  

- Profile the customers that responded to the campaign to understand the characteristics of customers who made purchases. 

- Develop models using the campaign results to target responsive, profitable customers for future campaigns. 

- Summarize and categorize the profitability of the various marketing segments which may be used to optimize future campaigns.

---

## Scientific questions/investigation

Given marketing dataset, there are three key problems to solve.  

- Given campaign period data, looking at customer profile, we aim to predict whether customer will Buy or Not Buy. We intent to frame this as a Classification problem. 

- Among the customers who are predicted as they will buy, we aim to estimate what will be size of purchase. We intent to frame this as a Regression problem. 

- For those customers who are likely to purchase and in general other customers using their historical product purchase data, we aim to predict top 3 products that are likely to be purchased. Also try to predict whether there are any product bundles Office Corp may sell more, etc. We intend to frame this as a Clustering problem.  

In the process of analysis, we intent to carry out,  

- data missing and imputation treatments,  

- data sufficiency and randomized splitting for test-train sets,  

- test amount of data necessary for optimal predictions & accuracy, etc.  

This last topic of determining optimal amount of data is called the ‘sample complexity study’ and it is extremely important for such real-life marketing campaign problems because collecting too much data unnecessarily may cause the company cost overrun. Data scientists should be able to advise the business department on how much data to collect to achieve a decent model performance. 

---

## Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
import matplotlib as mpl
mpl.rcParams['figure.dpi']=125

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

## Load (cleaned and formatted) dataset

In [ ]:
df = pd.read_csv("AddedFeatures_campaign_sale.csv")

In [ ]:
df.info()

In [ ]:
df.drop(['Index'],axis=1,inplace=True)

In [ ]:
df.columns

In [ ]:
cat_vars = ['Customer_engagement_length',
       'Language_group', 'Repurchase Method', 'Last Transaction Channel',
       'Number of Employees', 'Service Level', 'Do No Disturb',
       'Email Available', 'Desk', 'Executive Chair', 'Standard Chair',
       'Monitor', 'Printer', 'Computer', 'Insurance', 'Toner',
       'Office Supplies']

In [ ]:
d=pd.get_dummies(df,columns=cat_vars)

In [ ]:
d.info()

## Classification

In [ ]:
names = ["Logistic Regression","Nearest Neighbors", "Linear SVM", "RBF SVM", 
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost", "Bagging",
         "Naive Bayes", "QDA"]

### List of classifier (Scikit-learn estimator objects with hyperparamemetr settings)

In [ ]:
classifiers = [
    LogisticRegression(C=0.1,n_jobs=-1),
    KNeighborsClassifier(10,n_jobs=-1),
    SVC(kernel="linear", C=0.1),
    SVC(gamma='scale', C=1),
    DecisionTreeClassifier(max_depth=10,min_samples_leaf=10),
    RandomForestClassifier(max_depth=3, n_estimators=50, 
                           max_features=5,min_samples_leaf=10,n_jobs=-1),
    MLPClassifier(hidden_layer_sizes=(100,100),alpha=0.2, 
                  max_iter=200,learning_rate_init=0.01,learning_rate='adaptive',
                 early_stopping=True,validation_fraction=0.2),
    AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=3),
                       n_estimators=50,learning_rate=0.1),
    BaggingClassifier(estimator=DecisionTreeClassifier(max_depth=3),
                       n_estimators=50, max_features=5,n_jobs=-1),
    GaussianNB(),
    QuadraticDiscriminantAnalysis(reg_param=0.1)]

### Test/train set

In [ ]:
X = d.drop(['Campaign Period Sales','Buy'],axis=1)
y = d['Buy']

In [ ]:
TEST_FRAC = 0.2

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=TEST_FRAC, 
                                                    random_state=7406)

In [ ]:
X_train = StandardScaler().fit_transform(X_train)
X_test = StandardScaler().fit_transform(X_test)

### Running through the classifiers once

In [ ]:
for name, clf in zip(names, classifiers):
    t1 = time.time()
    clf.fit(X_train, y_train)
    t2 = time.time()
    delta_t = round((t2-t1)*1000,3)
    score = round(clf.score(X_test, y_test),3)
    print(f"Fitting with {name} took {delta_t} ms.\n Score: {score}")
    print("-"*75)

### Function to run through classifiers repeatedly

In [ ]:
def run_classifiers(clf_lst,names=None,num_runs=10, verbose=0):
    """
    Runs the list of classifiers for a fixed number of times
    """
    if names is None:
        names = [str(type(c)).split('.')[-1][:-2] for c in clf_lst]
    scores = dict.fromkeys(names,[])
    f1_scores = dict.fromkeys(names,[])
    runtimes = dict.fromkeys(names,[])
    for name, clf in zip(names, clf_lst):
        sc,f1,rt= [],[],[]
        for i in range(num_runs):
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_FRAC,)
            X_train = StandardScaler().fit_transform(X_train)
            X_test = StandardScaler().fit_transform(X_test)
            t1 = time.time()
            clf.fit(X_train, y_train)
            t2 = time.time()
            delta_t = round((t2-t1)*1000,3)
            score = round(clf.score(X_test, y_test),3)
            f1score = f1_score(y_test,clf.predict(X_test))
            sc.append(score)
            f1.append(f1score)
            rt.append(delta_t)
        sc = np.array(sc)
        f1 = np.array(f1)
        rt = np.array(rt)
        scores[name] = sc
        f1_scores[name] = f1
        runtimes[name] = rt
        if verbose:
            print(f"Finished {num_runs} runs for {name} algorithm")
            print("-"*75)
    # Convert to DataFrame
    df_scores = pd.DataFrame(scores)
    df_f1scores = pd.DataFrame(f1_scores)
    df_runtimes = pd.DataFrame(runtimes)
    
    return df_scores, df_f1scores,df_runtimes

### Run through clasifiers and plot

In [ ]:
d1,d2,d3 = run_classifiers(clf_lst=classifiers,
                           names=names,
                           num_runs=25,
                           verbose=1)

In [ ]:
d1

In [ ]:
def plot_bars(d,
              t1="Mean accuracy score of algorithms",
              t2="Std.dev of the accuracy scores of algorithms"):
    """
    """
    fig,ax=plt.subplots(1,2,figsize=(14,5))
    ax[0].barh(y=list(d.columns),width=d.describe().T['mean'],height=0.6,color='goldenrod')
    ax[0].set_title(t1)
    ax[1].barh(y=list(d.columns),width=d.describe().T['std'],height=0.6,color='dodgerblue')
    ax[1].set_title(t2)
    ax[0].spines['top'].set_visible(False)
    ax[0].spines['right'].set_visible(False)
    ax[0].spines['left'].set_visible(False)
    ax[0].spines['bottom'].set_color('#DDDDDD')
    ax[1].spines['top'].set_visible(False)
    ax[1].spines['right'].set_visible(False)
    ax[1].spines['left'].set_visible(False)
    ax[1].spines['bottom'].set_color('#DDDDDD')
    plt.tight_layout(pad=1.5)
    plt.show()

In [ ]:
mpl.rcParams.update(mpl.rcParamsDefault)
mpl.rcParams['xtick.labelsize']=13
mpl.rcParams['ytick.labelsize']=13
mpl.rcParams['figure.dpi']=125
mpl.rcParams['axes.titlesize']=18

In [ ]:
plot_bars(d1,
          t1="Mean accuracy score of algorithms",
          t2="Std.dev of the accuracy scores of algorithms")

In [ ]:
plot_bars(d2,
          t1="Mean F1-score of algorithms",
          t2="Std.dev of the F1-scores of algorithms")

In [ ]:
plot_bars(d3,
          t1="Mean training time of algorithms",
          t2="Std.dev of the training time of algorithms")

## Grid search of `AdaBoost`

In [ ]:
from sklearn.model_selection import GridSearchCV
abc = AdaBoostClassifier(estimator=DecisionTreeClassifier(), algorithm='SAMME')
parameters = {'estimator__max_depth':[i for i in range(2,11,2)],
              'estimator__min_samples_leaf':[5,10],
              'n_estimators':[10,50,250,1000],
              'learning_rate':[0.01,0.1]}
clf = GridSearchCV(abc, parameters,verbose=3,scoring='f1',n_jobs=-1)

In [ ]:
clf.fit(X_train,y_train)

In [ ]:
boost_grid=pd.DataFrame(clf.cv_results_['params'])

In [ ]:
boost_grid['F1-score']=clf.cv_results_['mean_test_score']

In [ ]:
boost_grid

In [ ]:
fig, ax = plt.subplots(2,2,figsize=(6,4))
ax= ax.ravel()
for i,c in enumerate(boost_grid.columns[:-1]):
    ax[i].scatter(boost_grid[c],boost_grid['F1-score'],
                 c='blue',edgecolor='k',alpha=0.7,s=150,
                 )
    ax[i].set_title(f"F1-score vs. {c}",fontsize=12)
    ax[i].set_ylim(0.5,0.8)
    ax[i].spines['top'].set_visible(False)
    ax[i].spines['right'].set_visible(False)
    ax[i].grid(True)
plt.tight_layout(pad=1.5)
plt.show()

## Regression

In [ ]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor

### List of regressors (Scikit-learn estimator objects with hyperparamemetr settings)

In [ ]:
reg_names = ["Linear regression","L1 (LASSO) regression","Ridge regression",
            "Support vector regression","Decision tree regression",
            "Random forest regression","Neural network regression"]

In [ ]:
regressors = [LinearRegression(n_jobs=-1),
              Lasso(alpha=0.1),
              Ridge(alpha=0.1),
              SVR(kernel='poly',degree=3),
              DecisionTreeRegressor(max_depth=10,min_samples_leaf=10),
              RandomForestRegressor(max_depth=3, n_estimators=50, 
                           max_features=5,min_samples_leaf=10,n_jobs=-1),
              MLPRegressor(hidden_layer_sizes=(100,100),alpha=0.2, 
                  max_iter=200,learning_rate_init=0.01,learning_rate='adaptive',
                 early_stopping=True,validation_fraction=0.2)]

### Regression data and test/train split

In [ ]:
d_reg= d[d['Campaign Period Sales']>0.0]

In [ ]:
X = d_reg.drop(['Campaign Period Sales','Buy'],axis=1)
y = d_reg['Campaign Period Sales']

### Function to run through classifiers repeatedly

In [ ]:
def run_regressors(reg_lst,names=None,num_runs=10, verbose=0):
    """
    Runs the list of regressors for a fixed number of times
    """
    if names is None:
        names = [str(type(c)).split('.')[-1][:-2] for c in reg_lst]
    scores = dict.fromkeys(names,[])
    runtimes = dict.fromkeys(names,[])
    for name, reg in zip(names, reg_lst):
        sc,rt= [],[]
        for i in range(num_runs):
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_FRAC,)
            X_train = StandardScaler().fit_transform(X_train)
            X_test = StandardScaler().fit_transform(X_test)
            t1 = time.time()
            reg.fit(X_train, y_train)
            t2 = time.time()
            delta_t = round((t2-t1)*1000,3)
            rmse = round(np.sqrt(np.mean((reg.predict(X_test)-y_test)**2).mean()),3)
            sc.append(rmse)
            rt.append(delta_t)
        sc = np.array(sc)
        rt = np.array(rt)
        scores[name] = sc
        runtimes[name] = rt
        if verbose:
            print(f"Finished {num_runs} runs for {name} algorithm")
            print("-"*75)
    # Convert to DataFrame
    df_scores = pd.DataFrame(scores)
    df_runtimes = pd.DataFrame(runtimes)
    
    return df_scores,df_runtimes

### Run through regressors and plot

In [ ]:
d1_reg, d2_reg = run_regressors(reg_lst=regressors,names=reg_names,num_runs=5,verbose=1)

In [ ]:
d1_reg

In [ ]:
d1_reg1 = d1_reg.drop(['Linear regression'],axis=1)

In [ ]:
d1_reg1['Neural network regression'].mean()

In [ ]:
plot_bars(d1_reg1,
          t1="RMSE score of algorithms",
          t2="Std.dev of the RMSE scores of algorithms")

In [ ]:
plot_bars(d2_reg,
          t1="Mean training time of algorithms",
          t2="Std.dev of the training time of algorithms")

In [ ]:
fig, ax = plt.subplots()
plt.hist(y,bins=100,edgecolor='k')
plt.xlim(0,2000)
plt.vlines(x=398,ymin=0,ymax=600,color='k',linestyle='--',linewidth=3)
plt.vlines(x=d1_reg1['Neural network regression'].mean(),
           ymin=0,ymax=600,color='orange',linestyle='--',linewidth=3)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.grid(True)
plt.show()

### Regression prediction and ground truth match plot

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_FRAC,)
X_train = StandardScaler().fit_transform(X_train)
X_test = StandardScaler().fit_transform(X_test)
nn_reg = MLPRegressor(hidden_layer_sizes=(100,100),alpha=0.2, 
                  max_iter=200,learning_rate_init=0.01,learning_rate='adaptive',
                 early_stopping=True,validation_fraction=0.2)
nn_reg.fit(X_train,y_train)
preds = nn_reg.predict(X_test)

In [ ]:
fig, ax = plt.subplots()
plt.scatter(x=y_test,y=preds,edgecolor='k',alpha=0.7,c='blue')
plt.plot(y_test,y_test,c='red')
plt.xlim(0,5000)
plt.ylim(0,5000)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.grid(True)
plt.xlabel("True campaign sales",fontsize=15)
plt.ylabel("Predicted campaign sales",fontsize=15)
plt.show()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_FRAC,)
xlim = y_test.max()
X_train = StandardScaler().fit_transform(X_train)
X_test = StandardScaler().fit_transform(X_test)

fig, ax = plt.subplots(2,3,figsize=(15,10))
ax= ax.ravel()
for i in range(6):
    reg = regressors[1:][i]
    reg.fit(X_train,y_train)
    preds = reg.predict(X_test)
    ax[i].scatter(x=y_test,y=preds,edgecolor='k',alpha=0.7,c='blue')
    ax[i].plot(y_test,y_test,c='red')
    ax[i].set_xlim(0,xlim*1.1)
    ax[i].set_ylim(0,xlim*1.1)
    ax[i].spines['top'].set_visible(False)
    ax[i].spines['right'].set_visible(False)
    ax[i].grid(True)
    ax[i].set_xlabel("True campaign sales",fontsize=15)
    ax[i].set_ylabel("Predicted campaign sales",fontsize=15)
    ax[i].set_title(reg_names[1:][i])
plt.tight_layout(pad=1.5)
plt.show()

## NN model tuning for regression

In [ ]:
no_neurons = [i*10 for i in range(1,11)]
alpha = [0.01,0.1,0.2,0.5]
learning_rate = [0.01,0.05,0.1]
activation = ['relu','tanh','logistic']

In [ ]:
nn_grid = {'neurons':[],'alpha':[],'learning_rate':[],'activation':[],'RMSE':[],'training-time':[]}

In [ ]:
nn_grid

In [ ]:
X = d_reg.drop(['Campaign Period Sales','Buy'],axis=1)
y = d_reg['Campaign Period Sales']

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=TEST_FRAC, 
                                                    random_state=7406)
X_train = StandardScaler().fit_transform(X_train)
X_test = StandardScaler().fit_transform(X_test)

In [ ]:
from tqdm import tqdm

for n in tqdm(no_neurons):
    for a in tqdm(alpha):
        for l in learning_rate:
            for act in activation:
                reg = MLPRegressor(hidden_layer_sizes=(n,n),alpha=a,activation=act, 
                  max_iter=200,learning_rate_init=l,learning_rate='adaptive',
                 early_stopping=True,validation_fraction=0.2)
                t1 = time.time()
                reg.fit(X_train, y_train)
                t2 = time.time()
                delta_t = round((t2-t1)*1000,3)
                rmse = round(np.sqrt(np.mean((reg.predict(X_test)-y_test)**2).mean()),3)
                nn_grid['neurons'].append(n)
                nn_grid['alpha'].append(a)
                nn_grid['learning_rate'].append(l)
                nn_grid['activation'].append(act)
                nn_grid['RMSE'].append(rmse)
                nn_grid['training-time'].append(delta_t)

In [ ]:
nn_grid=pd.DataFrame(nn_grid)

In [ ]:
nn_grid.shape

In [ ]:
nn_grid.head()

In [ ]:
plt.scatter(x=[i for i in range(len(nn_grid))],y=nn_grid['RMSE'])
plt.ylim(0,600)
plt.show()

In [ ]:
fig, ax = plt.subplots(2,2,figsize=(12,10))
ax= ax.ravel()
for i,c in enumerate(nn_grid.columns[:4]):
    ax[i].scatter(nn_grid[c],nn_grid['RMSE'],edgecolor='k',alpha=0.7,c='blue')
    ax[i].set_title(f"RMSE vs. {c}")
    ax[i].grid(True)
    ax[i].spines['top'].set_visible(False)
    ax[i].spines['right'].set_visible(False)
plt.show()

## Sample complexity

In [ ]:
sample_complexity = dict.fromkeys(names,[])
frac = [0.1*i for i in range(1,11)]
for name,clf in list(zip(names,classifiers)):
    scores = []
    for f in frac:
        d_frac = d.sample(frac=f)
        X = d_frac.drop(['Campaign Period Sales','Buy'],axis=1)
        y = d_frac['Buy']
        TEST_FRAC = 0.2
        X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=TEST_FRAC,random_state=7406)
        X_train = StandardScaler().fit_transform(X_train)
        X_test = StandardScaler().fit_transform(X_test)
        t1 = time.time()
        clf.fit(X_train, y_train)
        t2 = time.time()
        delta_t = round((t2-t1)*1000,3)
        f1score = f1_score(y_test,clf.predict(X_test))
        scores.append(f1score)
    sample_complexity[name] = np.array(scores)
        #print(sample_complexity[name])
    print(f"Done for {name}")

In [ ]:
sample_complexity=pd.DataFrame(sample_complexity)

In [ ]:
fig,ax = plt.subplots(4,3,figsize=(16,16),sharey=True)
ax = ax.ravel()
for i,c in enumerate(sample_complexity.columns):
    ax[i].plot([int(j) for j in range(1,11)],sample_complexity[c],marker='o',color='blue')
    ax[i].set_xticks([int(j) for j in range(1,11,2)])
    ax[i].set_xticklabels([str(10*j)+'%' for j in range(1,11,2)])
    ax[i].set_title(f"{c}",fontsize=14)
    ax[i].grid(True)
    ax[i].spines['top'].set_visible(False)
    ax[i].spines['right'].set_visible(False)
plt.show()